In [1]:
import pandas as pd

In [2]:
ProbNonDefaultGivenScoreGreaterXAndRace = pd.read_csv("ProbNonDefaultGivenScoreGreaterXAndRace.csv")
ProbLoanReceiverIsGood = pd.read_csv("ProbLoanReceiverIsGood.csv")

# ProbOfBeingScore = ProbScoreEqualsXGivenRace

In [3]:
ProbOfBeingScore = (
    pd.read_csv("figure3A.csv")
    .set_index(["Score"]).diff().fillna(0) / 100
)
# ProbOfBeingScore.columns.name = "Demographic"
ProbOfBeingScore.columns = ProbOfBeingScore.columns.str.lower()
ProbOfBeingScore.index.name = "TransRisk Score"

In [5]:
ProbScoreEqualsXGivenRace = pd.read_csv("ProbScoreEqualsXGivenRace.csv").set_index("TransRisk Score")

In [7]:
for race in ["white", "black", "hispanic", "asian"]:
    print(race, (ProbOfBeingScore[race.lower()] - ProbScoreEqualsXGivenRace[race]).abs().sum())

white 5.692061405548898e-17
black 4.7488055154865094e-17
hispanic 5.800481622797449e-17
asian 5.117434254131581e-17


# This is where they differ by 0 when I export them to a CSV, if you do cmd+f on the DataSetCalculations file for 'ProbScoreEqualsXGivenRace Differ' you'll see that they differ by more than zero before they are exported to a csv

In [19]:
ProbFromOld = pd.read_csv('ProbScoreEqualsXGivenRace-old.csv')
ProbFromNew = pd.read_csv('ProbScoreEqualsXGivenRace.csv')

for race in ["White", "Black", "Hispanic", "Asian"]:
    print(race, (ProbFromOld[race.lower()] - ProbFromNew[race]).abs().sum())

White 0.0
Black 0.0
Hispanic 0.0
Asian 0.0


# ProbOfBeingGoodAtScore = ProbNonDefaultAndScoreEqualsXGivenRace

In [8]:
ProbOfBeingGoodAtScore = (
    pd.read_csv("PercentageOfGoodsAndBads.csv")
    .set_index(["TransRisk Score", "Demographic"])
    .unstack()["Good"]
)

In [13]:
pis = [0.759185,0.315164,0.550595,0.80066]
CumulativePercentageByDefaulters = pd.read_csv("Figure7A-fixed.csv")

sizes = [133165, 18274, 14702, 7906]
total = sum(sizes)
ProbOfBeingRace = pd.DataFrame(
    { 
    'Demographic' : ['white', 'black', 'hispanic', 'asian'],
    'P(race)' : [sizes[0]/total, sizes[1]/total, sizes[2]/total, sizes[3]/total]
    },
    columns=["Demographic", "P(race)"]
)
ProbOfBeingRace.set_index('Demographic', inplace=True)
ProbRaceAndScoreEqualsX = pd.DataFrame({
    'white': ProbOfBeingRace.loc['white']['P(race)'] * ProbScoreEqualsXGivenRace['white'],
    'asian': ProbOfBeingRace.loc['asian']['P(race)'] * ProbScoreEqualsXGivenRace['asian'],
    'black': ProbOfBeingRace.loc['black']['P(race)'] * ProbScoreEqualsXGivenRace['black'],
    'hispanic': ProbOfBeingRace.loc['hispanic']['P(race)'] * ProbScoreEqualsXGivenRace['hispanic'],
})

def getProbDefOrNotGivenScoreEqualsX(dataset, names, pis):
    probabilities = pd.DataFrame(index=dataset.index)
    for i in range(len(names)):
        nonDefault = dataset[names[i] + ' (NonDefault)'].diff().fillna(value=0)
        default = dataset[names[i] + ' (Default)'].diff().fillna(value=0)
        finalProbability = (
            (pis[i] * nonDefault) / ((pis[i] * nonDefault) + ((1 - pis[i])*(default))))
        probabilities['P(NonDefault|Score=x, ' + names[i] + ')'] = finalProbability.fillna(value=0)
        probabilities['P(Default|Score=x, ' + names[i] + ')'] = (1 - finalProbability.fillna(value=0))
    probabilities['Score'] = dataset.index
    return probabilities.set_index('Score')

ProbDefOrNotGivenScoreEqualsXAndRace = getProbDefOrNotGivenScoreEqualsX(
    CumulativePercentageByDefaulters.set_index('Score'),
    ["White", "Black", "Hispanic", "Asian"],
    pis)

ProbNonDefaultGivenScoreEqualsXAndRace = (
    ProbDefOrNotGivenScoreEqualsXAndRace[['P(NonDefault|Score=x, White)', 
                                 'P(NonDefault|Score=x, Black)',
                                'P(NonDefault|Score=x, Hispanic)',
                                'P(NonDefault|Score=x, Asian)']])

ProbScoreEqualsXAndGoodAndRace = pd.DataFrame(index=ProbRaceAndScoreEqualsX.index)
ProbScoreEqualsXAndGoodAndRace['P(Score=x and NonDefault and White)'] = (
    ProbRaceAndScoreEqualsX['white'] * ProbNonDefaultGivenScoreEqualsXAndRace['P(NonDefault|Score=x, White)'])
ProbScoreEqualsXAndGoodAndRace['P(Score=x and NonDefault and Black)'] = (
    ProbRaceAndScoreEqualsX['black'] * ProbNonDefaultGivenScoreEqualsXAndRace['P(NonDefault|Score=x, Black)'])
ProbScoreEqualsXAndGoodAndRace['P(Score=x and NonDefault and Hispanic)'] = (
    ProbRaceAndScoreEqualsX['hispanic'] * ProbNonDefaultGivenScoreEqualsXAndRace['P(NonDefault|Score=x, Hispanic)'])
ProbScoreEqualsXAndGoodAndRace['P(Score=x and NonDefault and Asian)'] = (
    ProbRaceAndScoreEqualsX['asian'] * ProbNonDefaultGivenScoreEqualsXAndRace['P(NonDefault|Score=x, Asian)'])

ProbNonDefaultGivenScoreEqualsXAndRace.columns= ['asian', 'black', 'hispanic', 'white']

In [15]:
for race in ["white", "black", "hispanic", "asian"]:
    print(race, (ProbOfBeingGoodAtScore[race.lower()] - ProbNonDefaultGivenScoreEqualsXAndRace[race]).abs().sum())

white 8.502342054140064
black 1.6674386454431702
hispanic 0.4350214440731871
asian 8.92581709234939


# Did rest of the differ calculations in the DataSetCalculations workbook because I think reading them in as csvs is causing some numbers to be truncated or something?